In [2]:
import ipy_table
from collections import Counter
%run ../aux/postgres.py

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

#%run functions/preproc.py
#%run functions/classificacao.py

postgres functions loaded...


In [3]:
def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]

In [4]:
from nltk.stem.snowball import PortugueseStemmer
stemmer = PortugueseStemmer()
stemmer.stem("dogs")

print(stemmer.stem('gatos'))
print(stemmer.stem('gata'))
print(stemmer.stem('gato'))

gat
gat
gat


In [30]:
cmd = ("SELECT t.text, "
       "CASE "
       "WHEN cl.codclassificacao = 230 THEN 226 "
       "ELSE cl.codclassificacao END, "
       "CASE "
       "WHEN cl.codclassificacao = 230 THEN 'service' "
       "ELSE cl.descricao END "
       "FROM analisetweet at "
       "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
       "INNER JOIN tweet t on at.codtweet = t.codtweet "
       "WHERE codanaliseamostra = 126 AND at.codclassificacao <> 36 order by random()")

res01 = query(cmd)
topics = [x[2] for x in res01]
count01 = Counter(topics)

tot = 0
for x in count01.most_common():
    print(x)
    tot = tot + x[1]
print(tot)

tweets = [x[0] for x in res01]
print(len(tweets))

('service', 420)
('clinical', 172)
('personal', 138)
('product', 107)
('information', 94)
('sale', 66)
997
997


In [31]:
topicsID = [x[1] for x in res01]
tweets = [x[0] for x in res01]

print(topicsID[:10])

print(tweets[:10])
print(len(topicsID))
print(len(tweets))


[224, 226, 229, 226, 224, 224, 226, 229, 226, 223]
['Dental Implant machine w motor w reduction 16:1 push contra angle handpiece SET https://t.co/afHj5VEuga https://t.co/ZJOWzvs3dl', 'RT https://t.co/rvhOgYkKw2 Dental Implants are More Efficient than Traditional Dentures and Bridges. Explore details on https://t.co/PXAAhAgdDc #teeth #hornsbydentist #dentisthornsby #Oral… https://t.co/4GAfBK8nTi', 'Losing #teeth creates the potential for your oral #health to decline quickly…\n\nConsider a #dental implants: https://t.co/FCtOAjdmJP\n\n#tcmi https://t.co/vUapu9seeq', "We're on Pinterest!! \n\nDid you ever wonder what a dental implant looks like? ...or how Inv... https://t.co/LBqQ6fKLYA https://t.co/9Y8CQpQ538", 'Preferred Dental Technologies is the First Dental Implant Company to 3D Print Solid Custom ... https://t.co/Um9otgSCSS', 'RT @lintenaju7: Know More about Dental Implants and its Positive Features – Medaka Doctor https://t.co/hQhYkBsIQs', 'RT @olivesanders15: Orange County Dentist  

## Pré Proc

In [5]:
import re
happyFace = ['😅', '😁', '😍', '❤', '😂', '😋', '😌', '☺', '😘']
sadFace = ['😪', '😭', '😩']
neutralFace = ['😐', '😒']
scaryFace = ['😱']

def proc_emoji(tweet):
    for em in happyFace:
        tweet = tweet.replace(em, ' happyFace ')

    for em in neutralFace:
        tweet = tweet.replace(em, ' neutralFace ')

    for em in sadFace:
        tweet = tweet.replace(em, ' sadFace ')

    for em in scaryFace:
        tweet = tweet.replace(em, ' scaryFace ')
    return tweet
    
    
def pre_proc(doc_complete):
    doc_complete = [ t.lower() for t in doc_complete ]

    # Interrogações
    doc_complete = [re.sub("[$]", " moneymark ", x) for x in doc_complete ]

    # mention removal
    doc_complete = [re.sub("@\\w+", " mentionname ", x) for x in doc_complete ]
    # Remove URL
    doc_complete = [re.sub("(f|ht)(tp)(s?)(://)(.*)[.|/](.*)", " urlurl ", x) for x in doc_complete ]
    # Remove HTML symbols
    doc_complete = [re.sub("&[^\\s]*;", " ", x) for x in doc_complete ]
    # Remove numbers
    doc_complete = [re.sub("[0-9]+", " numbers ", x) for x in doc_complete ]
    # Interrogações
    doc_complete = [re.sub("[?]", " questionmark ", x) for x in doc_complete ]
    # Remove Punctuation    
    doc_complete = [re.sub("[.,\\/#!?+$–\"|%-\\^&\\*;:{}=\\-_`~()]", " ", x) for x in doc_complete ]
    # Remove Emojis
    doc_complete = [proc_emoji(x) for x in doc_complete ]
    
    pattern = re.compile('[\W_]+')
    doc_complete = [pattern.sub(' ', x) for x in doc_complete ]
    return doc_complete

In [32]:
tweetsPP = pre_proc(tweets)
for t, p in zip(tweets[:1000], tweetsPP[:1000]):
    print("tw: [%s]" % t)
    print("pp: [%s]" % p)
    print()

tw: [Dental Implant machine w motor w reduction 16:1 push contra angle handpiece SET https://t.co/afHj5VEuga https://t.co/ZJOWzvs3dl]
pp: [dental implant machine w motor w reduction numbers numbers push contra angle handpiece set urlurl ]

tw: [RT https://t.co/rvhOgYkKw2 Dental Implants are More Efficient than Traditional Dentures and Bridges. Explore details on https://t.co/PXAAhAgdDc #teeth #hornsbydentist #dentisthornsby #Oral… https://t.co/4GAfBK8nTi]
pp: [rt urlurl ]

tw: [Losing #teeth creates the potential for your oral #health to decline quickly…

Consider a #dental implants: https://t.co/FCtOAjdmJP

#tcmi https://t.co/vUapu9seeq]
pp: [losing teeth creates the potential for your oral health to decline quickly consider a dental implants urlurl tcmi urlurl ]

tw: [We're on Pinterest!! 

Did you ever wonder what a dental implant looks like? ...or how Inv... https://t.co/LBqQ6fKLYA https://t.co/9Y8CQpQ538]
pp: [we re on pinterest did you ever wonder what a dental implant looks like


tw: [Important information for anyone deciding on whether or not to get dental implants. https://t.co/s9Re3LzbV7]
pp: [important information for anyone deciding on whether or not to get dental implants urlurl ]

tw: [@cvpayne I got 10 percent discount on a dental implant for cash.]
pp: [ mentionname i got numbers percent discount on a dental implant for cash ]

tw: [Mini #Dental Implants in Glendale, AZ
Our expert Prosthodontist can provide you the mini dental implants in Glendale. https://t.co/fHRbvTKAa3]
pp: [mini dental implants in glendale az our expert prosthodontist can provide you the mini dental implants in glendale urlurl ]

tw: [AZ Max Oral Surgery - Dental Implants https://t.co/sELZ79pnoc #azmax #oralsurgery #smiles #DentalImplants https://t.co/LG6oApEyoN]
pp: [az max oral surgery dental implants urlurl ]

tw: [RT https://t.co/H0amsWUTac THE USE OF STEM CELLS IN DENTAL IMPLANT SITE DEVELOPMENT #Dentist #dxb #Dubai #dentali… https://t.co/YNERaORONV]
pp: [rt urlurl ]

tw: [Ho

pp: [world dental implants and orthodontics congress numbers in august tokyo urlurl ]

tw: [Dentures With Dental Implants Leicester &amp; Loughborough https://t.co/iWv99XtdG6]
pp: [dentures with dental implants leicester loughborough urlurl ]

tw: [Have a missing tooth? Contact our office to learn about dental implants today! https://t.co/GxkcX6xHmd]
pp: [have a missing tooth questionmark contact our office to learn about dental implants today urlurl ]

tw: [Personal Care for Dental Implants. Find out more https://t.co/ZNcAsJzIpT #DentalImplants]
pp: [personal care for dental implants find out more urlurl ]

tw: ["Dental Implants" https://t.co/9yGviDjKUl]
pp: [ dental implants urlurl ]

tw: [Is there something about your smile you are less than proud of? Dental implants may be the solution! We will help you regain your confidence from your smile! https://t.co/1AYxej1aGq]
pp: [is there something about your smile you are less than proud of questionmark dental implants may be the solution

## Pré Proc 2

In [6]:
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()
stemmer.stem("dogs")

'dog'

In [8]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from collections import Counter

stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
#doc_complete = [ t[0] for t in tweets ]
#print(stop)

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop and len(i) > 3])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(stemmer.stem(word) for word in punc_free.split())
    #normalized = [ w for w in punc_free if len(w) > 3 ]
    #normalized = punc_free
    return normalized

In [33]:
doc_clean = [clean(doc).split() for doc in tweetsPP] 
#print(doc_clean[:20])


print(len(doc_clean)*0.4)
ws = []
for f in doc_clean:
    for w in f:
        ws.append(w)
cws = Counter(ws)
rWords = [ x[0] for x in cws.most_common() ]
for x in cws.most_common(30):
    print(x)


398.8
('urlurl', 847)
('number', 366)
('mentionnam', 259)
('questionmark', 165)
('teeth', 136)
('smile', 92)
('dentist', 75)
('miss', 73)
('tooth', 60)
('cost', 54)
('dentistri', 51)
('replac', 48)
('restor', 46)
('market', 40)
('patient', 37)
('know', 35)
('best', 35)
('need', 35)
('help', 34)
('dentur', 34)
('surgeri', 33)
('look', 31)
('call', 31)
('learn', 30)
('treatment', 30)
('dentalimpl', 30)
('blog', 29)
('cosmet', 28)
('offer', 28)
('find', 28)


In [34]:
tweetsClean = []
for t in doc_clean:
    tweetsClean.append(" ".join([i for i in t]))

In [35]:
tweetsPP = pre_proc(tweets)
for t, p, dc, tc in zip(tweets[:10], tweetsPP[:10], doc_clean[:10], tweetsClean[:10]):
    print("tw: [%s]" % t)
    print("pp: [%s]" % p)
    print("cl: [%s]" % dc)
    print("tc: [%s]" % tc)
    print()
#print(tweets)

tw: [Dental Implant machine w motor w reduction 16:1 push contra angle handpiece SET https://t.co/afHj5VEuga https://t.co/ZJOWzvs3dl]
pp: [dental implant machine w motor w reduction numbers numbers push contra angle handpiece set urlurl ]
cl: [['machin', 'motor', 'reduct', 'number', 'number', 'push', 'contra', 'angl', 'handpiec', 'urlurl']]
tc: [machin motor reduct number number push contra angl handpiec urlurl]

tw: [RT https://t.co/rvhOgYkKw2 Dental Implants are More Efficient than Traditional Dentures and Bridges. Explore details on https://t.co/PXAAhAgdDc #teeth #hornsbydentist #dentisthornsby #Oral… https://t.co/4GAfBK8nTi]
pp: [rt urlurl ]
cl: [['urlurl']]
tc: [urlurl]

tw: [Losing #teeth creates the potential for your oral #health to decline quickly…

Consider a #dental implants: https://t.co/FCtOAjdmJP

#tcmi https://t.co/vUapu9seeq]
pp: [losing teeth creates the potential for your oral health to decline quickly consider a dental implants urlurl tcmi urlurl ]
cl: [['lose', 'tee

## Testing scikit

In [9]:
import string
import numpy as np

from joblib import Parallel, delayed
import multiprocessing

from nltk.stem import PorterStemmer
from nltk import word_tokenize


from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import sklearn.model_selection
from sklearn.model_selection import cross_val_score

In [10]:
def train(classifier, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
    classifier.fit(X_train, y_train)
    print("Accuracy: %s" % classifier.score(X_test, y_test))
    return classifier

In [11]:
# MultinomialNB
MNBtrials = []
cont = 0
contAll = 0
allTrials = []

for i in range(0, 11, 1):
    alpha = i/10
    for j in range(5, 11, 1):
        max_df = j/10
        for k in range(1, 11, 1):
            min_df = k
            trial = Pipeline([
                ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
                ('classifier', MultinomialNB(alpha=alpha))
                ])
            desc = 'MNB [alpha: ' + str(alpha) + ' max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
            MNBtrials.append((cont,desc,trial))
            cont+=1
            allTrials.append((contAll,desc,trial))
            contAll+=1

for t in MNBtrials[:10]:
    print(t[0],t[1])
print(len(MNBtrials))
print('--')
print(MNBtrials[2][1])
print('--')
print(len(allTrials))
print(allTrials[0])
print(allTrials[659])

0 MNB [alpha: 0.0 max_df:0.5 min_df: 1]
1 MNB [alpha: 0.0 max_df:0.5 min_df: 2]
2 MNB [alpha: 0.0 max_df:0.5 min_df: 3]
3 MNB [alpha: 0.0 max_df:0.5 min_df: 4]
4 MNB [alpha: 0.0 max_df:0.5 min_df: 5]
5 MNB [alpha: 0.0 max_df:0.5 min_df: 6]
6 MNB [alpha: 0.0 max_df:0.5 min_df: 7]
7 MNB [alpha: 0.0 max_df:0.5 min_df: 8]
8 MNB [alpha: 0.0 max_df:0.5 min_df: 9]
9 MNB [alpha: 0.0 max_df:0.5 min_df: 10]
660
--
MNB [alpha: 0.0 max_df:0.5 min_df: 3]
--
660
(0, 'MNB [alpha: 0.0 max_df:0.5 min_df: 1]', Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocess

In [12]:
# Random Forest
RFtrials = []
cont = 0

for val in range(1, 41):
    for j in range(5, 11, 1):
        max_df = j/10
        for k in range(1, 11, 1):
            min_df = k
            trial = Pipeline([
                ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
                ('classifier', RandomForestClassifier(n_estimators=val,random_state=10)),
                ])
            desc = 'RF [val: ' + str(alpha) + ' max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
            RFtrials.append((cont,desc,trial))
            cont+=1
            allTrials.append((contAll,desc,trial))
            contAll+=1            

for t in RFtrials[:12]:
    print(t[0],t[1])
print(len(RFtrials))
print('--')
print(RFtrials[2][1])


#for val in range(1,41):
#    trial = Pipeline([
#        ('vectorizer', TfidfVectorizer(min_df=5, max_df=0.9)),
#        ('classifier', RandomForestClassifier(n_estimators=val)),
#    ])
#    trials.append(trial)
print('--')
print(len(allTrials))
print(allTrials[0])
print(allTrials[659])

0 RF [val: 1.0 max_df:0.5 min_df: 1]
1 RF [val: 1.0 max_df:0.5 min_df: 2]
2 RF [val: 1.0 max_df:0.5 min_df: 3]
3 RF [val: 1.0 max_df:0.5 min_df: 4]
4 RF [val: 1.0 max_df:0.5 min_df: 5]
5 RF [val: 1.0 max_df:0.5 min_df: 6]
6 RF [val: 1.0 max_df:0.5 min_df: 7]
7 RF [val: 1.0 max_df:0.5 min_df: 8]
8 RF [val: 1.0 max_df:0.5 min_df: 9]
9 RF [val: 1.0 max_df:0.5 min_df: 10]
10 RF [val: 1.0 max_df:0.6 min_df: 1]
11 RF [val: 1.0 max_df:0.6 min_df: 2]
2400
--
RF [val: 1.0 max_df:0.5 min_df: 3]
--
3060
(0, 'MNB [alpha: 0.0 max_df:0.5 min_df: 1]', Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2'

In [13]:
# SGDClassifier
SGDtrials = []
cont = 0
lr = 1
for i in range(1,10):
    lr = lr / 10

    for j in range(5, 11, 1):
        max_df = j/10
        for k in range(1, 11, 1):
            min_df = k

            # SGD
            trial = Pipeline([
                ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
                ('classifier', SGDClassifier(random_state=10)),
                ])
            desc = 'SGD [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ' learning_rate : ' +str(lr) + ']'
            SGDtrials.append((cont,desc,trial))
            cont+=1
            allTrials.append((contAll,desc,trial))
            contAll+=1            
            
for j in range(5, 11, 1):
    max_df = j/10
    for k in range(1, 11, 1):
        min_df = k

        # SGD
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', SGDClassifier(random_state=10)),
            ])
        desc = 'SGD [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        SGDtrials.append((cont,desc,trial))
        cont+=1
        allTrials.append((contAll,desc,trial))
        contAll+=1            
            
            
            
for t in SGDtrials:
    print(t[0],t[1])
print(len(SGDtrials))
print('--')
print(SGDtrials[2][1])


0 SGD [max_df:0.5 min_df: 1 learning_rate : 0.1]
1 SGD [max_df:0.5 min_df: 2 learning_rate : 0.1]
2 SGD [max_df:0.5 min_df: 3 learning_rate : 0.1]
3 SGD [max_df:0.5 min_df: 4 learning_rate : 0.1]
4 SGD [max_df:0.5 min_df: 5 learning_rate : 0.1]
5 SGD [max_df:0.5 min_df: 6 learning_rate : 0.1]
6 SGD [max_df:0.5 min_df: 7 learning_rate : 0.1]
7 SGD [max_df:0.5 min_df: 8 learning_rate : 0.1]
8 SGD [max_df:0.5 min_df: 9 learning_rate : 0.1]
9 SGD [max_df:0.5 min_df: 10 learning_rate : 0.1]
10 SGD [max_df:0.6 min_df: 1 learning_rate : 0.1]
11 SGD [max_df:0.6 min_df: 2 learning_rate : 0.1]
12 SGD [max_df:0.6 min_df: 3 learning_rate : 0.1]
13 SGD [max_df:0.6 min_df: 4 learning_rate : 0.1]
14 SGD [max_df:0.6 min_df: 5 learning_rate : 0.1]
15 SGD [max_df:0.6 min_df: 6 learning_rate : 0.1]
16 SGD [max_df:0.6 min_df: 7 learning_rate : 0.1]
17 SGD [max_df:0.6 min_df: 8 learning_rate : 0.1]
18 SGD [max_df:0.6 min_df: 9 learning_rate : 0.1]
19 SGD [max_df:0.6 min_df: 10 learning_rate : 0.1]
20 SGD [

In [14]:
# SGDClassifier
SGDtrials = []
LinearSVCtrials = []
SVCtrials = []
cont = 0

for j in range(5, 11, 1):
    max_df = j/10
    for k in range(1, 11, 1):
        min_df = k
        
        # SGD
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', SGDClassifier(random_state=10)),
            ])
        desc = 'SGD [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        SGDtrials.append((cont,desc,trial))
        allTrials.append((contAll,desc,trial))
        contAll+=1            
        
        
        # Linear SVC
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', LinearSVC()),
            ])
        desc = 'LinearSVC [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        LinearSVCtrials.append((cont,desc,trial))
        allTrials.append((contAll,desc,trial))
        contAll+=1            
        
        # SVC
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', SVC()),
            ])
        desc = 'SVC [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        SVCtrials.append((cont,desc,trial))
        allTrials.append((contAll,desc,trial))
        contAll+=1            
        
        
        cont+=1


for t in SGDtrials[:10]:
    print(t[0],t[1])
print(len(SGDtrials))
print('--')
print(SGDtrials[2][1])

print('\n\n')

for t in LinearSVCtrials[:10]:
    print(t[0],t[1])
print(len(LinearSVCtrials))
print('--')
print(LinearSVCtrials[2][1])

print('\n\n')


for t in SVCtrials[:10]:
    print(t[0],t[1])
print(len(SVCtrials))
print('--')
print(SVCtrials[2][1])

0 SGD [max_df:0.5 min_df: 1]
1 SGD [max_df:0.5 min_df: 2]
2 SGD [max_df:0.5 min_df: 3]
3 SGD [max_df:0.5 min_df: 4]
4 SGD [max_df:0.5 min_df: 5]
5 SGD [max_df:0.5 min_df: 6]
6 SGD [max_df:0.5 min_df: 7]
7 SGD [max_df:0.5 min_df: 8]
8 SGD [max_df:0.5 min_df: 9]
9 SGD [max_df:0.5 min_df: 10]
60
--
SGD [max_df:0.5 min_df: 3]



0 LinearSVC [max_df:0.5 min_df: 1]
1 LinearSVC [max_df:0.5 min_df: 2]
2 LinearSVC [max_df:0.5 min_df: 3]
3 LinearSVC [max_df:0.5 min_df: 4]
4 LinearSVC [max_df:0.5 min_df: 5]
5 LinearSVC [max_df:0.5 min_df: 6]
6 LinearSVC [max_df:0.5 min_df: 7]
7 LinearSVC [max_df:0.5 min_df: 8]
8 LinearSVC [max_df:0.5 min_df: 9]
9 LinearSVC [max_df:0.5 min_df: 10]
60
--
LinearSVC [max_df:0.5 min_df: 3]



0 SVC [max_df:0.5 min_df: 1]
1 SVC [max_df:0.5 min_df: 2]
2 SVC [max_df:0.5 min_df: 3]
3 SVC [max_df:0.5 min_df: 4]
4 SVC [max_df:0.5 min_df: 5]
5 SVC [max_df:0.5 min_df: 6]
6 SVC [max_df:0.5 min_df: 7]
7 SVC [max_df:0.5 min_df: 8]
8 SVC [max_df:0.5 min_df: 9]
9 SVC [max_df:0.5 m

In [15]:
print(len(SGDtrials))
print(len(LinearSVCtrials))
print(len(SVCtrials))
print(len(MNBtrials))
print(len(RFtrials))

allTrials = SGDtrials + LinearSVCtrials + SVCtrials + MNBtrials + RFtrials
print(len(allTrials))

60
60
60
660
2400
3240


In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import model_selection

from sklearn.metrics import  make_scorer
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=42)
model=allTrials[928][2]

#results = model_selection.cross_val_score(estimator=model,
#                                          X=features,
#                                          y=labels,
#                                          cv=kfold,
#                                          scoring='f1_micro'

In [17]:
#%%writefile functions/classificacao.py
def classifierScore(classifier, X_train, y_train, X, y, metrica, idx):
    
    scoring = {'accuracy' : make_scorer(accuracy_score), 
               'precision' : make_scorer(precision_score),
               'recall' : make_scorer(recall_score), 
               'f1' : make_scorer(f1_score)}
    scoring = ['accuracy', 'precision']
    scoring = 'f1_micro'

    kfold = model_selection.KFold(n_splits=10, random_state=42)
    
    #print(classifier[1])
    classifier[2].fit(X_train, y_train)
    scores = model_selection.cross_val_score(classifier[2], X, y, cv=kfold, scoring=metrica)
    return (idx, scores)

def bestClassifier(X, y, classifiers, metrica):
    target_names = list(set(y))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=50)
    maxScore = 0
    ic = 0
    
    
    #for idx, val in enumerate(allTrials):
        
    num_cores = multiprocessing.cpu_count()
    resultados = Parallel(n_jobs=num_cores)(delayed(classifierScore)(classifier, X_train, y_train, X, y, metrica, idx) 
                                            for idx, classifier in enumerate(classifiers))
                                            #for classifier in classifiers)
    #print(resultados)

    for result in resultados:
        classifier = classifiers[result[0]]

        scores = result[1]
        #print(scores)

        score = scores.mean()
        std = scores.std() * 2

        if score > maxScore:
            maxScore = score
            maxStd = std
            maxIc = ic
            bClassifier = classifier
        ic += 1

    print(bClassifier[1], '[', bClassifier[0], ']')
    print("max: ", maxScore)
    print("%s: %0.2f (+/- %0.2f)" % (metrica, maxScore, maxStd))
    
    print('----')
    
    y_true = []
    y_pred = []

    y_true = y_test
    classifiers[bClassifier[0]][2].fit(X_train, y_train)
    y_pred = classifiers[bClassifier[0]][2].predict(X_test)
    report2 = sklearn.metrics.classification_report(y_true, y_pred)
    print(report2)
    
    
    return bClassifier

# Melhor Classificador Dental Implants (tweets pré-processados):

In [18]:
print(len(allTrials))

3240


In [19]:
allTrials[111]

(51, 'LinearSVC [max_df:1.0 min_df: 2]', Pipeline(memory=None,
          steps=[('vectorizer',
                  TfidfVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.float64'>,
                                  encoding='utf-8', input='content',
                                  lowercase=True, max_df=1.0, max_features=None,
                                  min_df=2, ngram_range=(1, 1), norm='l2',
                                  preprocessor=None, smooth_idf=True,
                                  stop_words=None, strip_accents=None,
                                  sublinear_tf=False,
                                  token_pattern='(?u)\\b\\w\\w+\\b',
                                  tokenizer=None, use_idf=True,
                                  vocabulary=None)),
                 ('classifier',
                  LinearSVC(C=1.0, class_weight=None, dual=True,
               

In [26]:
a = bestClassifier(tweetsPP, topics, allTrials, 'f1_weighted')

LinearSVC [max_df:0.9 min_df: 1] [ 40 ]
max:  0.5289331688157526
f1_weighted: 0.53 (+/- 0.06)
----
              precision    recall  f1-score   support

    clinical       0.39      0.43      0.41        53
 information       0.46      0.44      0.45        25
    personal       0.55      0.52      0.53        44
     product       0.19      0.20      0.19        30
        sale       0.40      0.21      0.28        19
     service       0.63      0.65      0.64       129

    accuracy                           0.50       300
   macro avg       0.44      0.41      0.42       300
weighted avg       0.50      0.50      0.50       300



In [27]:
b = bestClassifier(tweets, topics, allTrials, 'f1_weighted')

LinearSVC [max_df:0.9 min_df: 1] [ 40 ]
max:  0.5360868507294835
f1_weighted: 0.54 (+/- 0.07)
----
              precision    recall  f1-score   support

    clinical       0.40      0.53      0.46        53
 information       0.46      0.48      0.47        25
    personal       0.67      0.50      0.57        44
     product       0.25      0.20      0.22        30
        sale       0.33      0.21      0.26        19
     service       0.64      0.67      0.66       129

    accuracy                           0.53       300
   macro avg       0.46      0.43      0.44       300
weighted avg       0.53      0.53      0.53       300



# Melhor Classificador Dental Implants (tweets pré-processados):

In [28]:
c = bestClassifier(tweetsPP, topics, allTrials, 'accuracy')

LinearSVC [max_df:1.0 min_df: 1] [ 50 ]
max:  0.5445151515151515
accuracy: 0.54 (+/- 0.06)
----
              precision    recall  f1-score   support

    clinical       0.40      0.47      0.43        53
 information       0.42      0.44      0.43        25
    personal       0.61      0.45      0.52        44
     product       0.19      0.23      0.21        30
        sale       0.25      0.16      0.19        19
     service       0.63      0.64      0.64       129

    accuracy                           0.50       300
   macro avg       0.42      0.40      0.40       300
weighted avg       0.50      0.50      0.50       300



In [29]:
d = bestClassifier(tweets, topics, allTrials, 'accuracy')

LinearSVC [max_df:0.9 min_df: 1] [ 40 ]
max:  0.5546363636363637
accuracy: 0.55 (+/- 0.06)
----
              precision    recall  f1-score   support

    clinical       0.40      0.53      0.46        53
 information       0.46      0.48      0.47        25
    personal       0.67      0.50      0.57        44
     product       0.25      0.20      0.22        30
        sale       0.33      0.21      0.26        19
     service       0.64      0.67      0.66       129

    accuracy                           0.53       300
   macro avg       0.46      0.43      0.44       300
weighted avg       0.53      0.53      0.53       300



In [38]:
allTrials[100]

(40, 'LinearSVC [max_df:0.9 min_df: 1]', Pipeline(memory=None,
          steps=[('vectorizer',
                  TfidfVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.float64'>,
                                  encoding='utf-8', input='content',
                                  lowercase=True, max_df=0.9, max_features=None,
                                  min_df=1, ngram_range=(1, 1), norm='l2',
                                  preprocessor=None, smooth_idf=True,
                                  stop_words=None, strip_accents=None,
                                  sublinear_tf=False,
                                  token_pattern='(?u)\\b\\w\\w+\\b',
                                  tokenizer=None, use_idf=True,
                                  vocabulary=None)),
                 ('classifier',
                  LinearSVC(C=1.0, class_weight=None, dual=True,
               

In [37]:
for idx, t in enumerate('allTrials'):
    if (t[0] == 40):
        print(idx)
        print(t)

40
(40, 'SGD [max_df:0.9 min_df: 1]', Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                ('classifier',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
            

In [31]:
allTrials

[(0, 'SGD [max_df:0.5 min_df: 1]', Pipeline(memory=None,
           steps=[('vectorizer',
                   TfidfVectorizer(analyzer='word', binary=False,
                                   decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, max_df=0.5, max_features=None,
                                   min_df=1, ngram_range=(1, 1), norm='l2',
                                   preprocessor=None, smooth_idf=True,
                                   stop_words=None, strip_accents=None,
                                   sublinear_tf=False,
                                   token_patt...
                  ('classifier',
                   SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                                 early_stopping=False, epsilon=0.1, eta0=0.0,
                                 fit_intercept=True, 

## Matriz de Confusão

In [122]:
yga = [ 0,1,2,3,4,5,6,7,8,9]
print(yga[:3])
print(yga[3:])

[0, 1, 2]
[3, 4, 5, 6, 7, 8, 9]


In [123]:
len(tweetsPP[:697])

697

In [144]:
a

(51, 'LinearSVC [max_df:1.0 min_df: 2]', Pipeline(memory=None,
      steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=2,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ax_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0))]))

In [57]:
bestTrial = allTrials[55]
bestTrial[2].fit(tweetsPP[:697], topics[:697])


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=6,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ty='l2', power_t=0.5, random_state=10, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [58]:
y_pred = bestTrial[2].predict(tweetsPP[697:])
for a in Counter(y_pred).most_common():
    print(a)

('service', 103)
('personal', 72)
('product', 58)
('information', 37)
('clinical', 16)
('sale', 14)


In [85]:
y_true = topics[697:]
for a in Counter(y_true).most_common():
    print(a)

('service', 83)
('service clinical', 50)
('personal', 44)
('clinical', 39)
('product', 31)
('information', 31)
('sale', 22)


### Opções:

- remover de vez as categorias
- buscar no total de tweets, mensagens que sejam classificadas nessas categorias




## Executando classificador em todos os tweets

In [39]:
# all tweets
cmd = ("select count(*) "
       "from amostratweet at "
       "inner join tweet t on at.codtweet = t.codtweet "
       "where codamostra = 219")
query(cmd)

[(293306,)]

In [20]:
# all tweets
cmd = ("select count(*) "
       "from amostratweet at "
       "inner join tweet t on at.codtweet = t.codtweet "
       "where codamostra = 219")
query(cmd)

[(299987,)]

In [21]:
cmd = ("select t.codtweet, t.text, 0 "
       "from amostratweet at "
       "inner join tweet t on at.codtweet = t.codtweet "
       "where codamostra = 219")
clTweets = query(cmd)

In [22]:
clTweets[:10]

[(77431828,
  'Immediate dental implant - Can you really do that? is a new blog on our website. It explains the procedure in... https://t.co/aD8ZWwryEc',
  0),
 (77431881, 'dental implants', 0),
 (77431997,
  'RT @TomMeyerDDS: Dental implants - what they are and why you may want to consider this procedure. https://t.co/66WhABA2s5 #DesPlaines #dent…',
  0),
 (77432011, 'The Versatility Of Dental Implants https://t.co/pXYknPe8M3', 0),
 (77432169,
  'Are you considering dental Implants, and have questions about the procedure?  Well this informative video may... https://t.co/woRCdat9UK',
  0),
 (77432237,
  'Dental Implants or Dentures &amp; Partials: Which Is Right for You? https://t.co/iWf6ZjpyWW',
  0),
 (77432239,
  'Dental Implants or Dentures &amp; Partials: Which Is Right for You? https://t.co/iWf6ZjpyWW https://t.co/4vK2HCMuMJ',
  0),
 (77432412,
  "Don't forget to register for the next Dental Implant Seminar. July 13 in East Aurora; July 14 in Amherst.... https://t.co/4N4qeo9ygw",

In [23]:
allTweets = []
for t in clTweets:
    allTweets.append(t[1])
allTweetsPP = pre_proc(allTweets)

In [24]:
doc_clean = [clean(doc).split() for doc in allTweetsPP] 
allTweetsClean = []
for t in doc_clean:
    allTweetsClean.append(" ".join([i for i in t]))

In [25]:
print(len(clTweets))
print(len(allTweets))
print(len(allTweetsPP))
print(len(doc_clean))
print(len(allTweetsClean))


for cl, t, p, dc, tc in zip(clTweets[:10], allTweets[:10], allTweetsPP[:10], doc_clean[:100], allTweetsClean[:100]):
    print("[%s]" % cl[0])
    print("[%s]" % cl[1])
    print("[%s]" % t)
    print("[%s]" % p)
    print("[%s]" % dc)
    print("[%s]" % tc)
    print('---\n')
#print(tweets)


299987
299987
299987
299987
299987
[77431828]
[Immediate dental implant - Can you really do that? is a new blog on our website. It explains the procedure in... https://t.co/aD8ZWwryEc]
[Immediate dental implant - Can you really do that? is a new blog on our website. It explains the procedure in... https://t.co/aD8ZWwryEc]
[immediate dental implant can you really do that questionmark is a new blog on our website it explains the procedure in urlurl ]
[['immedi', 'realli', 'questionmark', 'blog', 'websit', 'explain', 'procedur', 'urlurl']]
[immedi realli questionmark blog websit explain procedur urlurl]
---

[77431881]
[dental implants]
[dental implants]
[dental implants]
[[]]
[]
---

[77431997]
[RT @TomMeyerDDS: Dental implants - what they are and why you may want to consider this procedure. https://t.co/66WhABA2s5 #DesPlaines #dent…]
[RT @TomMeyerDDS: Dental implants - what they are and why you may want to consider this procedure. https://t.co/66WhABA2s5 #DesPlaines #dent…]
[rt mentionn

In [36]:
tweetsClean[:10]

['machin motor reduct number number push contra angl handpiec urlurl',
 'urlurl',
 'lose teeth creat potenti oral health declin quick consid urlurl tcmi urlurl',
 'pinterest ever wonder look like questionmark urlurl',
 'prefer technolog first compani number print solid custom urlurl',
 'mentionnam know posit featur medaka doctor urlurl',
 'mentionnam orang counti dentist provid patient qualiti general cosmet dentistri invisalig',
 'enhanc best smile post cnipo urlurl',
 'tooth loss happen smile hold back urlurl',
 'market project wit signific growth number radiant insight urlurl']

In [37]:
tweets[:10]

['Dental Implant machine w motor w reduction 16:1 push contra angle handpiece SET https://t.co/afHj5VEuga https://t.co/ZJOWzvs3dl',
 'RT https://t.co/rvhOgYkKw2 Dental Implants are More Efficient than Traditional Dentures and Bridges. Explore details on https://t.co/PXAAhAgdDc #teeth #hornsbydentist #dentisthornsby #Oral… https://t.co/4GAfBK8nTi',
 'Losing #teeth creates the potential for your oral #health to decline quickly…\n\nConsider a #dental implants: https://t.co/FCtOAjdmJP\n\n#tcmi https://t.co/vUapu9seeq',
 "We're on Pinterest!! \n\nDid you ever wonder what a dental implant looks like? ...or how Inv... https://t.co/LBqQ6fKLYA https://t.co/9Y8CQpQ538",
 'Preferred Dental Technologies is the First Dental Implant Company to 3D Print Solid Custom ... https://t.co/Um9otgSCSS',
 'RT @lintenaju7: Know More about Dental Implants and its Positive Features – Medaka Doctor https://t.co/hQhYkBsIQs',
 'RT @olivesanders15: Orange County Dentist  provides patients with quality general and co

In [26]:
allTrials[100]

(40, 'LinearSVC [max_df:0.9 min_df: 1]', Pipeline(memory=None,
          steps=[('vectorizer',
                  TfidfVectorizer(analyzer='word', binary=False,
                                  decode_error='strict',
                                  dtype=<class 'numpy.float64'>,
                                  encoding='utf-8', input='content',
                                  lowercase=True, max_df=0.9, max_features=None,
                                  min_df=1, ngram_range=(1, 1), norm='l2',
                                  preprocessor=None, smooth_idf=True,
                                  stop_words=None, strip_accents=None,
                                  sublinear_tf=False,
                                  token_pattern='(?u)\\b\\w\\w+\\b',
                                  tokenizer=None, use_idf=True,
                                  vocabulary=None)),
                 ('classifier',
                  LinearSVC(C=1.0, class_weight=None, dual=True,
               

In [38]:
bestTrial = allTrials[100]
bestTrial[2].fit(tweets, topics)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
   

In [39]:
y_pred = bestTrial[2].predict(allTweets)
for a in Counter(y_pred).most_common():
    print(a)

('service', 154619)
('clinical', 60886)
('personal', 34535)
('product', 22278)
('information', 18290)
('sale', 9379)


In [40]:
for a in Counter(y_pred).most_common():
    print(a[0],';',a[1])

service ; 154619
clinical ; 60886
personal ; 34535
product ; 22278
information ; 18290
sale ; 9379


In [41]:
len(y_pred)

299987

In [47]:
for cl, allt, yp in zip(clTweets, allTweetsClean, y_pred):
    #print("[%s]" % cl[0])
    #print("[%s]" % cl[1])
    #print("[%s]" % allt)
    #print("[%s]" % yp)
    
    codcl = 39
    if yp == 'information': codcl = 223
    if yp == 'product': codcl = 224
    if yp == 'service': codcl = 226
    if yp == 'personal': codcl = 227
    if yp == 'sale': codcl = 228
    if yp == 'clinical': codcl = 229
    if yp == 'service clinical' : codcl = 230
    
    #if codcl != 39: continue
    #print("[%s]" % dc)
    #print("[%s]" % tc)
    cmd = ("INSERT INTO resultadoanaliseprojetotweet "
           "(codresultadoanaliseprojeto, codclassificacao, codtweet) VALUES " 
           "(20, " + str(codcl) + " , " + str(cl[0]) + ")");
    #print(cmd)
    execSQL(cmd)
    
    #print('---\n')
#print(tweets)


In [46]:
query("select * from resultadoanaliseprojeto")

[(2, 19, 45, 'filtro 1'),
 (3, 19, 47, 'filtro 1 '),
 (4, 2, 20, 'india filtro 2'),
 (5, 3, 20, 'uk filtro 2'),
 (6, 4, 20, 'usa filtro 2'),
 (1, 1, 40, 'Filtro 1'),
 (7, 1, 40, 'Filtro 6'),
 (8, 26, 78, None),
 (9, 26, 79, None),
 (19, 28, 124, 'ML Classification 2019 update'),
 (11, 31, 116, 'Original conciliação Debora / Dayane / Geraldine'),
 (10, 31, 108, 'Revisão Geraldine pós conciliação'),
 (12, 28, 124, None),
 (14, 28, 124, 'LDA: Service 30 topics'),
 (13, 28, 124, 'LDA: Information 30 topics'),
 (15, 28, 124, 'LDA: Clinical'),
 (17, 28, 124, 'LDA: Product'),
 (18, 28, 124, 'LDA: Sale'),
 (16, 28, 124, 'LDA: Personal'),
 (20, None, None, 'Dental Health December 2020')]

In [48]:
cmd = ("SELECT "
       "cl.descricao, "
       "to_char(datatweet, 'YYYY-MM') as data, "
       "count(r.codtweet) "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "INNER JOIN tweet t ON r.codtweet = t.codtweet "
       "where codresultadoanaliseprojeto = 20 "
       "GROUP BY 1, 2 "
       "ORDER BY 1, 2"
      )
res = query(cmd)
for r in res:
    print(r[0],";",r[1],";",r[2])

clinical ; 2016-07 ; 1265
clinical ; 2016-08 ; 2055
clinical ; 2016-09 ; 1625
clinical ; 2016-10 ; 2834
clinical ; 2016-11 ; 1351
clinical ; 2016-12 ; 1710
clinical ; 2017-01 ; 4012
clinical ; 2017-02 ; 2315
clinical ; 2017-03 ; 1831
clinical ; 2017-04 ; 1353
clinical ; 2017-05 ; 1543
clinical ; 2017-06 ; 1816
clinical ; 2017-07 ; 1887
clinical ; 2017-08 ; 1564
clinical ; 2017-09 ; 1660
clinical ; 2017-10 ; 1772
clinical ; 2017-11 ; 1534
clinical ; 2017-12 ; 1702
clinical ; 2018-01 ; 1544
clinical ; 2018-02 ; 1443
clinical ; 2018-03 ; 1310
clinical ; 2018-04 ; 1010
clinical ; 2018-05 ; 210
clinical ; 2018-07 ; 257
clinical ; 2018-08 ; 638
clinical ; 2018-09 ; 1383
clinical ; 2018-10 ; 1731
clinical ; 2018-11 ; 1110
clinical ; 2018-12 ; 1241
clinical ; 2019-01 ; 1964
clinical ; 2019-02 ; 3011
clinical ; 2019-03 ; 1799
clinical ; 2019-04 ; 1398
clinical ; 2019-05 ; 1252
clinical ; 2019-06 ; 1260
clinical ; 2019-07 ; 1313
clinical ; 2019-08 ; 860
clinical ; 2019-09 ; 934
clinical ; 2019-1